# Opschonen shapefiles

Dit notebook schoont de shapefiles op waar nodig. Het past ook omzettabellen toe bij de karteringen waar dat nodig is (Alde Feanen en Grote Wielen). 

De karteringen en enkele bestanden moeten eerst van de Spheer OneDrive gehaald worden. Instructies:

 - Kopieer de provinciefolders uit de OneDrive: `Spheer AI/Spheer AI - General/Projecten/Veg_2_Hab/Data/Vegetatiekarteringen` naar `testing/vegetatiekarteringen` (of naar iets anders, dan moet je `originele_karteringen_prefix` aanpassen onder **config**)
 - `Overzicht_vegetatiekarteringen.xlsx`, de andere overzicht xlxs en de omzettabellen/. staan in `Spheer AI\Spheer AI - General\Projecten\Veg_2_Hab\Data\bestanden_alle_karteringen_door_veg2hab.ipynb`. Kopieer deze naar `testing/`.
- Geef in `opgeschoonde_shapefiles_prefix` de naam van de folder waar de opgeschoonde shapefiles moeten komen. Dit is ook het folder waaruit de shapefilekarteringen moeten worden geladen in het `alle_karteringen_door_veg2hab.ipynb` notebook.
- Run de rest van de cellen. Deze zullen eerst alle karteringen overkopieren, en daarna de op te schonen karteringen overschrijven met een opgeschoonde versie.

### Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import os

originele_karteringen_prefix = '../testing/vegetatiekarteringen/'
opgeschoonde_shapefiles_prefix = '../testing/opgeschoonde_shapefiles/'

if not os.path.exists(opgeschoonde_shapefiles_prefix):
    os.makedirs(opgeschoonde_shapefiles_prefix)

### Inladen benodigde informatie


In [ ]:
# info over de karteringen
overzicht = pd.read_excel(
    "../testing/Overzicht_vegetatiekarteringen.xlsx", 
    sheet_name="Vegetatiekarteringen", 
    engine='openpyxl',
    usecols=[
        "provincie", 
        "naam_kartering", 
        "path_shapes", 
        "landelijke_typologie beschikbaar?", 
        "locatie_landelijke_typologie",
    ],
    dtype="string"
)

# benodigde info voor de shapefile karteringen
shape_info = pd.read_excel(
    "../testing/shapefilekarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "ElmID_col",
        "datum_col",
        "opmerking_col",
        "vegtype_col_format",
        "welke_typologie",
        "SBB_col",
        "VvN_col",
        "split_char",
        "perc_col",
    ],
    dtype="string"
)
# Convert all pd.NA to None to prevent having to do "if not pd.isna()" everywhere
# NOTE: dit naar vegkaartering.py?
shape_info = shape_info.applymap(lambda x: None if pd.isna(x) else x)

overzicht = overzicht[overzicht["landelijke_typologie beschikbaar?"] == "Ja"]
overzicht.path_shapes = overzicht.path_shapes.apply(lambda x: originele_karteringen_prefix + x[2:])

def read_kartering(name):
    row = overzicht[overzicht.naam_kartering == name].iloc[0]
    path = row.path_shapes
    gdf = gpd.read_file(path)
    return path, gdf

def save_kartering(path, gdf):
    path = path.replace(originele_karteringen_prefix, opgeschoonde_shapefiles_prefix)
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    gdf.to_file(path)

### Kopieren van de karteringen

Alles onveranderd overzetten

In [ ]:
for naam in list(overzicht.naam_kartering):
    path, gdf = read_kartering(naam)
    save_kartering(path, gdf)
    print(f"Copied {naam} to {path}")

### Overschrijven van de op te schonen karteringen

In [ ]:
path, gdf = read_kartering("BG_Holtingerveld_bossen2016")

# omwisselen van omgekeerde SBB en VvN
mask = gdf["OBJECTID_1"].isin([359, 24, 166, 215])
gdf.loc[mask, "F1_VvN"], gdf.loc[mask, "F1_SBB"] = gdf.loc[mask, "F1_SBB"], gdf.loc[mask, "F1_VvN"]

# "-" in perc_col vervangen door None
perc_cols = ["F1_Perc", "F2_Perc", "F3_Perc", "F4_Perc"]
for col in perc_cols:
    gdf[col] = gdf[col].apply(lambda x: None if x == "-" else x)

# save
save_kartering(path, gdf)

In [ ]:
path, gdf = read_kartering("HDL_meerdere_terreinen2016")

# Weghalen " RG" suffix van CODE_SBB
gdf["CODE_SBB"] = gdf["CODE_SBB"].str.replace(" RG", "")

# save
save_kartering(path, gdf)

In [ ]:
path, gdf = read_kartering("HDL_meerdere_terreinen2017")

# Vervangen van "-" en "?" door None in VGTYPE_VvN
# NOTE: Bij de vraagtekens staat in de opmerking dat het onduidelijk is, moeten we daar wat mee?
gdf["VGTYPE_VvN"] = gdf["VGTYPE_VvN"].apply(lambda x: None if x == "?" else x)

# save
save_kartering(path, gdf)

In [ ]:
path, gdf = read_kartering("NM_Fochteloerveen2013_14")

# Vervangen van "nvt" en "vegetatieloos" door None in VVNCode
gdf["VVNCode"] = gdf["VVNCode"].apply(lambda x: None if x in ["nvt", "vegetatieloos"] else x)

# save
save_kartering(path, gdf)

In [ ]:
path, gdf = read_kartering("SBB0756_Drentsche_A2009")

# Vervangen van 50A+ door 50A in SBBTYPE1
gdf["SBBTYPE1"] = gdf["SBBTYPE1"].apply(lambda x: "50A" if x == "50A+" else x)

# save
save_kartering(path, gdf)

In [ ]:
path, gdf = read_kartering("Mandefjild_2015")

# Omwisselen 2_SBB2 en 2_VVN1 waar VLAKNR_DEF 1017, 1072, 1037, 1038, 1079, 1077 is
mask = gdf["VLAKNR_DEF"].isin([1017, 1072, 1037, 1038, 1079, 1077])
gdf.loc[mask, "2_VVN1"], gdf.loc[mask, "2_SBB2"] = gdf.loc[mask, "2_SBB2"], gdf.loc[mask, "2_VVN1"]

# Omwisselen (3_SBB1 en 3_SBB2) en (3_VVN1 en 3_VVN2) waar VLAKNR_DEF 1111
mask = gdf["VLAKNR_DEF"].isin([1111])
gdf.loc[mask, "3_SBB1"], gdf.loc[mask, "3_SBB2"] = gdf.loc[mask, "3_SBB2"], gdf.loc[mask, "3_SBB1"]
gdf.loc[mask, "3_VVN1"], gdf.loc[mask, "3_VVN2"] = gdf.loc[mask, "3_VVN2"], gdf.loc[mask, "3_VVN1"]

# "11Aa2c" in 1_SBB1 vervangen door "11A2c" (Staat ook als VvN 11Aa2a in de shapefile)
gdf["1_SBB1"] = gdf["1_SBB1"].apply(lambda x: "11A2c" if x == "11Aa2c" else x)

# "36Ab" in 2_SBB1 vervangen door "36A-b" (Staat ook als VvN 36RG1 in de shapefile)
gdf["2_SBB1"] = gdf["2_SBB1"].apply(lambda x: "36A-b" if x == "36Ab" else x)

# "41RG/DG" in 1_VVN1 vervangen door "41"
gdf["1_VVN1"] = gdf["1_VVN1"].apply(lambda x: "41" if x == "41RG/DG" else x)

# save
save_kartering(path, gdf)

In [ ]:
path, gdf = read_kartering("KetlikerSkar_2016")

# Vervangen van "PteAqu" en "0" door None in VegCode
gdf["VegCode"] = gdf["VegCode"].apply(lambda x: None if x in ["PteAqu", "0"] else x)

# save
save_kartering(path, gdf)

In [ ]:
path, gdf = read_kartering("Bosgroep_SNLMonitoring_2014")

# Vervangen van "open water" en "zand" door None in CODE_SBB
# NOTE: Willen we dit vertalen naar de goeie 50A/50B/50C?
gdf["CODE_SBB"] = gdf["CODE_SBB"].apply(lambda x: None if x in ["open water", "zand"] else x)

# save
save_kartering(path, gdf)

### Omzettabellen

In [ ]:
omzettabel_grote_wielen = pd.read_excel(
    "../testing/omzettabel_grote_wielen.xlsx",
    engine='openpyxl',
    sheet_name="Omzettabel",
    usecols=[
        "LokaalType",
        "SBB Jordy",
    ],
    dtype="string"
)

omzettabel_alde_feanen = pd.read_excel(
    "../testing/omzettabel_alde_feanen.xlsx",
    engine='openpyxl',
    sheet_name="Bewerkt door Jakko",
    usecols=[
        "LokaalType",
        "SBB",
    ],
    dtype="string"
)

In [ ]:
alde_feanen_path, alde_feanen_gdf = read_kartering("Alde_Feanen_2012")
grote_wielen_path, grote_wielen_gdf = read_kartering("Grote_Wielen_2013")

### Alde Feanen
# Laten vallen van nutteloze kolommen (er zijn er anders echt heel veel)
alde_feanen_gdf = alde_feanen_gdf[["ElmID", "LBLKORT", "DATUM", "Opmerking", "geometry"]]

# Splitten op "/" en door de omzettabel halen
alde_feanen_gdf["LBLKORT"] = alde_feanen_gdf["LBLKORT"].apply(lambda typ_str: [omzettabel_alde_feanen[omzettabel_alde_feanen.LokaalType == lok_typ].SBB.iloc[0] if lok_typ in omzettabel_alde_feanen.LokaalType.values else None for lok_typ in typ_str.split("/")] if typ_str is not None else None)

# None waardes weghalen
alde_feanen_gdf["LBLKORT"] = alde_feanen_gdf["LBLKORT"].apply(lambda l: [x for x in l if x is not None] if l is not None else None)

# Joinen met "+"
alde_feanen_gdf["LBLKORT"] = alde_feanen_gdf["LBLKORT"].apply(lambda l: "+".join(l) if l is not None else None)

### Grote Wielen
# Door de omzettabel halen
for col in ["type1", "type2", "type3", "type4", "type5"]:
    grote_wielen_gdf[col] = grote_wielen_gdf[col].apply(
        lambda lok_typ: omzettabel_grote_wielen[omzettabel_grote_wielen.LokaalType == lok_typ]["SBB Jordy"].iloc[0] if lok_typ is not None else None
    )

### Save
save_kartering(alde_feanen_path, alde_feanen_gdf)
save_kartering(grote_wielen_path, grote_wielen_gdf)